## Raster Data Processing with Python

### Acquire data using GeoWeb Services 
 - __WCS (Web Coverage Service)__ <br>
WCS is a standard OGC Standard, which is supported by GeoServer, an open-source geoservices software, and ArcGIS Server. WCS offers multi-dimensional coverage data for access over the Internet.
 - __WMS (Web Map Service) and WMTS (Web Map Tile Service)__
WMS and WMTS are also OGC standards. They serve pre-rendered images over the Internet, but those images are used for display in the browser, not suitable for further analysis.

#### Use OWSLib Library

For more information, see [this tutorial on Raster Handling with Python](https://geoscripting-wur.github.io/PythonRaster/) and [the OWSLib GitHub](https://github.com/geopython/OWSLib) and [Documentation](https://geopython.github.io/OWSLib/).
<p>
Unfortunately, it seems the MLC WCS service does not respond to request at this moment. Most applications use WMS, instead.

In [ ]:
import requests
import json
import geopandas as gpd
import fiona 
from owslib.wcs import WebCoverageService

US Land Cover Data are available through MRLC (Multi-resolution Land Characteristics Consortium)
https://www.mrlc.gov/data-services-page

In [ ]:
lyrs = fiona.listlayers('./us_census.gpkg')
print(lyrs)

In [ ]:
# Load the shapefile and get the bounding box
aoi = gpd.read_file('./us_census.gpkg', layer = lyrs[1], driver = 'GPKG').query('NAME == "Kings"')
bbox = aoi.to_crs('epsg:4326').bounds.values[0]; # 5070 is used by NLCD
print(bbox)

In [ ]:
wcs = WebCoverageService('https://www.mrlc.gov/geoserver/mrlc_download/NLCD_2019_Land_Cover_L48/wcs?service=WCS', version='2.0.1')
print(list(wcs.contents))

In [ ]:
print([op.name for op in wcs.operations])

In [ ]:
# Take the 0.5m DSM as an example
cvg = wcs.contents['mrlc_download__NLCD_2019_Land_Cover_L48']

# Print supported reference systems, the bounding box defined in WGS 84 coordinates, and supported file formats
print(wcs.contents.keys())
print(wcs['mrlc_download__NLCD_2019_Land_Cover_L48'].id)
print(cvg.boundingboxes)
print(cvg.supportedFormats)

In [ ]:
# Define a bounding box in the available crs (see before) by picking a point and drawing a 1x1 km box around it
# bbox = aoi.to_crs('epsg:5070').bounds.values[0]
#bbox = (x - 500, y - 500, x + 500, y + 500)

# Request the DSM data from the WCS
response = wcs.getCoverage(identifier=[wcs['mrlc_download__NLCD_2019_Land_Cover_L48'].id], 
                            bbox=bbox, 
                            format='image/tiff',
                            crs='urn:ogc:def:crs:EPSG::4326',
                            width=200, height=200, timeout = 120)

# Write the data to a local file in the 'data' directory
with open('mrlc_nlcd_2019_nys.tif', 'wb') as file:
    file.write(response.read())


#### Use Plain HTTP Request

Not recommended. Not working either. See [an online example](https://medium.com/@caserobertson/python-national-land-cover-database-nlcd-example-ea11408eaef5
).

In [ ]:
# Load the shapefile and get the bounding box
import requests
import json
import geopandas as gpd

# Define the API endpoint and parameters
url = 'https://www.mrlc.gov/geoserver/wcs'
params = {
    'service': 'wcs',
    'version': '1.1.1',
    'request': 'GetCoverage',
    'identifier': 'NLCD_2016_Land_Cover_L48',
    'format': 'image/tiff',
    'scaleFactor': '1',
    'interpolation': 'nearest-neighbor'
}

# Load the shapefile and get the bounding box
bbox = aoi.to_crs('epsg:4326').bounds.values[0]

# Add the subset parameter to the API request
# params['crs'] = "EPSG:4326"
params['BoundingBox'] = f'{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]},EPSG:4326'

# Send the request to the API endpoint
response = requests.get(url, params=params)

# Save the response to a file
with open('nlcd_data.tif', 'wb') as f:
    f.write(response.content)


### Read Local Raster Files

As directly reading raster data using WCS is not quite reliable, we often use local files for raster data.



#### One band land cover data
The data was downloaded from MRLC website. It has been clipped to New York State. The pixels have also been reclassifed to have two binary values for residential/developed land vs others.

In [ ]:
import rasterio 
import numpy as np
import pandas as pd
from rasterio import plot as rioplot
from matplotlib import pyplot as plt

In [ ]:
src = rasterio.open("./ny_lc/LU_NY.tif", driver='GTIFF')
print(src)

In [ ]:
def printRasterAttribute(rstDat):
    print("Dataset Name: {}. Band number {}\n \
       Image width: {}, Image Height: {} \n \
       Band Indexes:{}, Band data type: {} \n\
       Image CRS: {} with bounds of \n{}".format(rstDat.name, rstDat.count, 
       rstDat.width, rstDat.height,
       rstDat.indexes, rstDat.dtypes,
       rstDat.crs, rstDat.bounds))

In [ ]:
printRasterAttribute(src)

In [ ]:
# Read all bands, with be a three dimensional ndarray
bands = src.read()
print(list(bands.shape))

In [ ]:
# We can directly access the band using numpy methods
print(list(bands[0].shape))

In [ ]:
bands[0][0:10, 0:10]

In [ ]:
# Read a specific band
# Note the band index base is one, instead of zero
band1 = src.read(1)

plt.imshow(band1, cmap='pink')
plt.show()

In [ ]:
print(type(band1))
print(list(band1.shape))

For basic `numpy` operations, see [the cheat sheet](https://assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf).

In [ ]:
def printBandAttribute(bandDat):
    print('Dimension of the ndarray is {}. \n \
            Data Type is {}.\n \
            min is {}, max is {}, \n \
            unique values: {}'.format(
        list(bandDat.shape), 
        bandDat.dtype,
        bandDat.min(),
        bandDat.max(),
        np.unique(bandDat)))


In [ ]:
printBandAttribute(band1)

In [ ]:
printBandAttribute(bands[0])

In [ ]:
rioplot.show_hist(band1, bins=64)

In [ ]:
rioplot.show(band1)

In [ ]:
band1[band1 == 127] = 0
rioplot.show(band1)

In [ ]:
printBandAttribute(band1)

In [ ]:
# We can covert the image to an array, then to a single column of pandas DataFrame
pd.DataFrame(band1.ravel()).describe()

In [ ]:
# What does this do?
sumStats = {str(x):len(band1[band1 == x]) for x in np.unique(band1)}
print(sumStats)

#### Multiband Image

The image is downloaded from NAIP (National Agricultural Imagery Program).

In [ ]:
naip = rasterio.open("./ny_lc/m_4007305_sw_18_1_20170809.tif")

In [ ]:
printRasterAttribute(naip) # four bands of (R, G, B, NIR)

For raster plotting, see [this tutorial using both `matplotlib` and `rasterio`](http://patrickgray.me/open-geo-tutorial/chapter_3_visualization.html) and [the `rasterio` plot documentation](https://rasterio.readthedocs.io/en/stable/topics/plotting.html) 

In [ ]:
rasterio.plot.show(naip)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,6))

rioplot.show_hist(naip, bins=50, stacked = True, 
                    alpha = 0.8, histtype='barstacked', 
                    title="Histogram", lw=0.0,
                    ax = ax)

ax.legend(['nir', 'red', 'green','blue'])

In [ ]:
rasterio.plot.show(naip.read(),naip.transform)


In [ ]:
fig, ax = plt.subplots(1, figsize=(12,12))
# For colormaps, see https://matplotlib.org/stable/tutorials/colors/colormaps.html
# use _r to the named colormap to reverse the color sequences
naipimg = rioplot.show((naip, 4), cmap='summer_r', adjust= True, ax=ax)
#fig.colorbar(naipimg, ax=ax)
#rioplot.show((naip, 4), contour=True, ax=ax)

As each band can be read in as numpy ndarray, we can make band-based calculations or map algebra easily.

In [ ]:
bandRed = naip.read(1)
bandNIR = naip.read(4)
ndvi = (bandNIR.astype(float)-bandRed.astype(float))/(bandNIR.astype(float)+bandRed.astype(float))


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8)) 

# Show the color image
rioplot.show(naip, ax=axs[0], title='Natural Color')
rioplot.show(ndvi, ax=axs[1], cmap='RdYlGn', title="NDVI")
